In [ ]:
! unzip /content/drive/MyDrive/covid-dataset.zip -d /content/coviddataset/

Streaming output truncated to the last 5000 lines.
  inflating: /content/coviddataset/Dataset/train/Normal/Normal-5812.png  
  inflating: /content/coviddataset/Dataset/train/Normal/Normal-5813.png  
  inflating: /content/coviddataset/Dataset/train/Normal/Normal-5814.png  
  inflating: /content/coviddataset/Dataset/train/Normal/Normal-5815.png  
  inflating: /content/coviddataset/Dataset/train/Normal/Normal-5816.png  
  inflating: /content/coviddataset/Dataset/train/Normal/Normal-5817.png  
  inflating: /content/coviddataset/Dataset/train/Normal/Normal-5818.png  
  inflating: /content/coviddataset/Dataset/train/Normal/Normal-5819.png  
  inflating: /content/coviddataset/Dataset/train/Normal/Normal-582.png  
  inflating: /content/coviddataset/Dataset/train/Normal/Normal-5820.png  
  inflating: /content/coviddataset/Dataset/train/Normal/Normal-5821.png  
  inflating: /content/coviddataset/Dataset/train/Normal/Normal-5822.png  
  inflating: /content/coviddataset/Dataset/train/Normal/Normal

#Import required libraries for building the model and handling images.

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Input

from tensorflow.keras.models import Model

from tensorflow.keras.preprocessing import image

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input

from glob import glob

import numpy as np

import matplotlib.pyplot as plt

#Set the image size to [299, 299] pixels and specify paths for training and testing datasets.

In [ ]:
imagesize=[299, 299]

trainPath= r"/content/coviddataset/Dataset/train"




testPath= r"/content/coviddataset/Dataset/test"



#Create data generators for the training and testing datasets. The images are rescaled, and augmentation techniques like shear, zoom, and horizontal flip are applied to the training images.

In [ ]:
train_datagen= ImageDataGenerator(rescale = 1./255,
                                    shear_range = 0.2,
                                    zoom_range = 0.2,
                                    horizontal_flip=True)

test_datagen=ImageDataGenerator (rescale = 1./255)


In [ ]:
training_set = train_datagen. flow_from_directory('/content/coviddataset/Dataset/train',
                                                    target_size=(299, 299),
                                                    batch_size = 16,
                                                    class_mode= 'categorical')

test_set = test_datagen.flow_from_directory('/content/coviddataset/Dataset/test',
                                                target_size=(299, 299),
                                                batch_size = 16,
                                                class_mode= 'categorical')

Found 19159 images belonging to 4 classes.
Found 2006 images belonging to 4 classes.


#Load the Xception model pretrained on the ImageNet dataset and modify it for transfer learning. The top classification layers are removed, and a custom classification head is added. The model summary is displayed.

In [ ]:
from tensorflow.keras.applications import Xception
xception = Xception(input_shape=imagesize + [3], weights='imagenet', include_top=False)

83683744/83683744 [==============================] - 3s 0us/step


In [ ]:
for layer in xception.layers:
  layer.trainable = False

In [ ]:
x = Flatten()(xception.output)

In [ ]:
prediction = Dense(4,activation='softmax')(x)

In [ ]:
model = Model(inputs=xception.input, outputs=prediction)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 149, 149, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                             

#Compile the model with categorical cross-entropy loss, Adam optimizer, and accuracy metric.

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

#Define a callback to monitor and stop training early if no improvement is observed.

In [ ]:
import tensorflow
my_callbacks=[
    tensorflow.keras.callbacks.EarlyStopping (patience=6)
]

In [ ]:
print(len(training_set))

1198


#Train the model using the fit_generator function. Specify the training and validation data, number of epochs, and steps per epoch for both sets. The defined early stopping callback is utilized during training.

In [ ]:
r = model.fit_generator(
    training_set,
    validation_data=test_set,
    epochs=25,
    steps_per_epoch=len(training_set)//16,  # Adjusted value
    validation_steps=len(test_set)//16,  # Adjusted value
    callbacks=my_callbacks
)

<ipython-input-15-d143b3756e17>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/25
74/74 [==============================] - 42s 403ms/step - loss: 4.0590 - accuracy: 0.6748 - val_loss: 3.6147 - val_accuracy: 0.7143
Epoch 2/25
74/74 [==============================] - 29s 391ms/step - loss: 3.8785 - accuracy: 0.7382 - val_loss: 10.4551 - val_accuracy: 0.6071
Epoch 3/25
74/74 [==============================] - 31s 414ms/step - loss: 4.1410 - accuracy: 0.7280 - val_loss: 6.9682 - val_accuracy: 0.6875
Epoch 4/25
74/74 [==============================] - 29s 391ms/step - loss: 4.3238 - accuracy: 0.7365 - val_loss: 5.3039 - val_accuracy: 0.7768
Epoch 5/25
74/74 [==============================] - 29s 397ms/step - loss: 3.8299 - accuracy: 0.7542 - val_loss: 5.5470 - val_accuracy: 0.7589
Epoch 6/25
74/74 [==============================] - 29s 387ms/step - loss: 3.7135 - accuracy: 0.7711 - val_loss: 8.1893 - val_accuracy: 0.7054
Epoch 7/25
74/74 [==============================] - 29s 392ms/step - loss: 3.9900 - accuracy: 0.7660 - val_loss: 4.6843 - val_accuracy: 0.758

##Import the EarlyStopping and ModelCheckpoint callbacks from Keras.Define the callbacks: EarlyStopping to stop training when validation accuracy doesn't improve, and ModelCheckpoint to save the best model based on validation accuracy.Train the model again using the defined callbacks.

In [ ]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
es = EarlyStopping(monitor= "val_accuracy" , min_delta= 0.01, patience= 3, verbose=1)
mc = ModelCheckpoint(filepath="bestmodel.h5", monitor="val_accuracy", verbose=1, save_best_only= True)

In [ ]:
hist = model.fit(training_set,
    validation_data=test_set,
    epochs=25,
    steps_per_epoch=len(training_set)//16,  # Adjusted value
    validation_steps=len(test_set)//16,
    callbacks=[es,mc]
                 )

Epoch 1/25
74/74 [==============================] - ETA: 0s - loss: 3.5813 - accuracy: 0.7956
Epoch 1: val_accuracy did not improve from 0.81641
74/74 [==============================] - 33s 406ms/step - loss: 3.5813 - accuracy: 0.7956 - val_loss: 3.8217 - val_accuracy: 0.8036
Epoch 2/25
74/74 [==============================] - ETA: 0s - loss: 3.4862 - accuracy: 0.7838
Epoch 2: val_accuracy did not improve from 0.81641
74/74 [==============================] - 29s 387ms/step - loss: 3.4862 - accuracy: 0.7838 - val_loss: 6.8937 - val_accuracy: 0.7500
Epoch 3/25
74/74 [==============================] - ETA: 0s - loss: 3.1718 - accuracy: 0.8074
Epoch 3: val_accuracy did not improve from 0.81641
74/74 [==============================] - 29s 387ms/step - loss: 3.1718 - accuracy: 0.8074 - val_loss: 4.6240 - val_accuracy: 0.7411
Epoch 4/25
74/74 [==============================] - ETA: 0s - loss: 2.7988 - accuracy: 0.8032
Epoch 4: val_accuracy improved from 0.81641 to 0.82143, saving model to bes

#Save the final trained model to a file.

In [ ]:
model.save("/content/drive/MyDrive/model/model.h5")

#Import necessary libraries for handling images and predictions.Load the trained model saved earlier.Load an image for prediction, preprocess it, and convert it into the required shape.Make a prediction using the loaded model and identify the most likely class label.

In [ ]:
import cv2

from tensorflow.keras.utils import load_img, img_to_array

from tensorflow.keras.applications. inception_v3 import decode_predictions

from tensorflow.keras.models import load_model
from tensorflow.keras.applications.xception import Xception

In [ ]:
model=load_model("/content/drive/MyDrive/model/model.h5")

In [ ]:
model=load_model("/content/bestmodel.h5")

In [ ]:
img = '/content/coviddataset/Dataset/test/Lung_Opacity/Lung_Opacity-5627.png'

image = load_img(img, target_size=(299,299))

image=img_to_array(image)

image=image.reshape((1,299,299,3))

image.shape

(1, 299, 299, 3)

In [ ]:
pred=model.predict(image)

1/1 [==============================] - 0s 26ms/step


In [ ]:
pred

array([[1., 0., 0., 0.]], dtype=float32)

In [ ]:
labels = ['COVID','Lung_opacity','Normal','Viral Pneumonia']

In [ ]:
import numpy as np

p=np.argmax(pred)


In [ ]:
p

0

#Print the predicted label for the given image.


In [ ]:

labels[p]

'COVID'